In [1]:
import requests
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np

In [2]:
sia=SentimentIntensityAnalyzer()

In [3]:
dataframe=pd.DataFrame(columns=['NAME','TITLE','POLARITY','REVIEW','TYPE_OF_TRAVELLER','SEAT_TYPE','ROUTE','DATE_FLOWN',
                               'SEAT_COMFORT','CABIN_STAFF_SERVICE','FOOD_AND_BEVERAGES','INFLIGHT_ENTERTAINMENT',
                               'GROUND_SERVICE','WIFI_AND_CONNECTIVITY','VALUE_FOR_MONEY','RECOMMENDED'])

In [4]:
std={
    'TYPE_OF_TRAVELLER':'Type Of Traveller',
    'SEAT_TYPE':'Seat Type',
    'ROUTE':'Route',
    'DATE_FLOWN':'Date Flown',
    'SEAT_COMFORT':'Seat Comfort',
    'CABIN_STAFF_SERVICE':'Cabin Staff Service',
    'FOOD_AND_BEVERAGES':'Food & Beverages',
    'INFLIGHT_ENTERTAINMENT':'Inflight Entertainment',
    'GROUND_SERVICE':'Ground Service',
    'WIFI_AND_CONNECTIVITY':'Wifi & Connectivity',
    'VALUE_FOR_MONEY':'Value For Money',
    'RECOMMENDED':'Recommended'
}
site="https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/"
for i in range (1,357):
    url=site.format(i=i)
    page=requests.get(url)
    soup=BeautifulSoup(page.content,"html.parser")
    results=soup.find_all(class_="body")
    for result in results:
        header=result.find(class_="text_header").text
        name=result.find(itemprop="name").text
        review=result.find(itemprop="reviewBody").text
        polarity=sia.polarity_scores(review)['compound']
        table=result.find(class_="review-ratings").find_all('tr')
        stats={}
        for row in table:
            cols=row.find_all('td')
            val=[]
            for elem in cols:
                val.append(elem.text.strip())
                try:
                    if 'review-rating-stars' in elem.get('class'):
                        val.append(elem.find_all(class_='star fill')[-1].text)
                except IndexError:
                    val.append(np.nan)
            if(len(val)>2):
                del val[1]
            stats[val[0]]=val[1]
        if(list(std.values()) != list(stats.keys())):
            li=[t for t in list(std.values()) if t not in list(stats.keys())]
            for col in li:
                stats[col]=np.nan
        rev='POSITIVE'
        if(polarity<=0):
            rev='NEGATIVE'
        dataframe.loc[len(dataframe.index)]=[name,
                                                header,
                                                polarity,
                                                rev,
                                                stats[std['TYPE_OF_TRAVELLER']],
                                                stats[std['SEAT_TYPE']],
                                                stats[std['ROUTE']],
                                                stats[std['DATE_FLOWN']],
                                                stats[std['SEAT_COMFORT']],
                                                stats[std['CABIN_STAFF_SERVICE']],
                                                stats[std['FOOD_AND_BEVERAGES']],
                                                stats[std['INFLIGHT_ENTERTAINMENT']],
                                                stats[std['GROUND_SERVICE']],
                                                stats[std['WIFI_AND_CONNECTIVITY']],
                                                stats[std['VALUE_FOR_MONEY']],
                                                stats[std['RECOMMENDED']]
                                            ]


In [5]:
dataframe

,NAME,TITLE,POLARITY,REVIEW,TYPE_OF_TRAVELLER,SEAT_TYPE,ROUTE,DATE_FLOWN,SEAT_COMFORT,CABIN_STAFF_SERVICE,FOOD_AND_BEVERAGES,INFLIGHT_ENTERTAINMENT,GROUND_SERVICE,WIFI_AND_CONNECTIVITY,VALUE_FOR_MONEY,RECOMMENDED
0,B Sherry,"""cancel your flight without notice""",-0.9379,NEGATIVE,Couple Leisure,Premium Economy,Dallas to Madrid via London,May 2023,1,1,1,1,1,1,1,no
1,William Jackson,"""flights changed with no cost""",-0.6711,NEGATIVE,Couple Leisure,Economy Class,London to Valencia,March 2023,4,4,NaN,NaN,5,NaN,4,yes
2,A Warten,"""Cheap, quick and efficient""",0.8479,POSITIVE,Solo Leisure,Economy Class,London to Glasgow,May 2023,5,5,5,NaN,5,NaN,5,yes
3,E Michaels,"""the worst major European airline""",-0.1289,NEGATIVE,Business,Business Class,London Heathrow to Malaga,May 2023,2,2,3,1,1,1,1,no
4,Steve Bennett,"""do not think the fare was worth the money""",0.9763,POSITIVE,Couple Leisure,Business Class,London to Nice,May 2023,2,3,3,NaN,4,1,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3548,C Mcculloch,British Airways customer review,-0.6562,NEGATIVE,NaN,Economy Class,NaN,NaN,3,3,3,4,NaN,NaN,3,no
3549,Nick Berry,British Airways customer review,0.8720,POSITIVE,NaN,Business Class,NaN,NaN,4,5,4,NaN,NaN,NaN,3,yes
3550,Avril Barclay,British Airways customer review,0.4516,POSITIVE,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,yes
3551,C Volz,British Airways customer review,0.9148,POSITIVE,NaN,Premium Economy,NaN,NaN,1,3,5,NaN,NaN,NaN,1,no


In [7]:
dataframe.to_excel('../Extracted-Data/dataset.xlsx',index=False)